# Introduction

In this work, we are going to build joint alpha factors from the research paper "The Formation Process of Winners and Losers in Momentum Investing". 

The paper develops a model showing that past returns and the formation process of past returns have joint effect on future expected returns. That means, the past behavior of a stock that has become a winner (bullish) or a looser (bearish) plays an important role in momentum investing. 

The paper has shown that as long as two stocks share the same return over the past $n-$month, the future expected return of the stock whose historical prices are convex shaped is not lower than the one with concave-shaped historical prices. 

In a more concrete terms, stocks with convex-shaped historical prices will possess higher future expected returns than the one with concave-shaped historical prices.

A polynomial quandratic model is is used to approximate the nonlinear pattern of historical prices. 

In what follows, we are going to explore the paper to build a joint alpha factor for predicting future returns based on he formation patterns of past returns. The resulting alpha factor will be evaluated with performance metrics including information coefficient, turnover, sharpe ratio, and so on.



## Mathematical formulation of winners and loosers  concepts from market prices


First of all we define the notion of convextiy and concavity:

A real valuedfunction $f$ is convex in an interval $\mathbb{D}$, if for every $\lambda \in [0, 1]$,

$$f(\lambda x + (1-\lambda y) ) \leq \lambda f(x) + (1-\lambda) f(y)\quad \forall x, y \in \mathbb{D} $$

A real valuedfunction $f$ is convex in an interval $\mathbb{D}$, if for every $\lambda \in [0, 1]$,

$$f(\lambda x + (1-\lambda y) ) \geq \lambda f(x) + (1-\lambda) f(y)\quad \forall x, y \in \mathbb{D} $$

$\textbf{ Properties}:$

1. $f$ is convex if and only if $f^{\prime\prime}(x) \geq 0 \quad \forall x\in \mathbb{D}$.
2. $f$ is concave if and only if $f^{\prime\prime}(x) \leq 0 \quad \forall x\in \mathbb{D}$.



$\textbf{ Example:}$
1. $f(x) = x^2$ is convex.
2. $f(x)=\sqrt{x}$ is concave.

### Approximating curves with polynomials
The stock's closing price curves are approximated with polynomials of the form $$y(t) = \beta t + \gamma t^2$$
where $t$ represents the number of days from the start of the stocks trajectory and $y(t)$ is the stock closing price at time $t$.

From the polynomial approximation, we can verify whether the approximation is convex or concave using $\textbf{Properties 1. or 2.}$, respectively.

### Accelerated Gain and Loss coefficients  from stock price's curvature

From now on, the polynomial approximation is re-written  by specifying the meaning of its coefficients. That is 
$$y(t) = \text{gain}\times t + \text{acceleration} \times t^2$$

The coefficient $\textbf{gain}$ represents the slop of the linear term of the polynomial. The linear coefficient primarily affects the tilt or slope of the curve, especially near the origin. Hence it captures the upward or downward trend of the stock price depending on whether it is posive or negative. 



The coefficient  $\textbf{acceleration}$ in front of the quadratic term $t^2$ determines the curvature of the graph. This is the "convexity" or "concavity" of the curve:
 1. If $\textbf{acceleration} > 0$, the curve is convex (or concave up), meaning it opens upwards. This is because as $t$ increases, the $t^2$ term, being squared, always adds a positive contribution to $y(t)$, pulling the curve upwards as t $t$ moves away from the origin. Therefore, if $\textbf{gain} > 0$, the trend of the stock is upward convex and reflects an **accelerated gain**, whereas if $\textbf{gain} < 0$, the trend of the stock is downward convex and reflects an **decelerated loss**. 
     
 2. If $\textbf{acceleration} < 0$, the curve is concave (or concave down), meaning it opens downwards. In this case, the contribution of the $t^2$ term is negative, pushing the curve downwards as $t$ moves away from the origin. Therefore, if $\textbf{gain} > 0$, the trend of the stock is downward concave and reflects a **decelerated gain**, whereas if $\textbf{gain} < 0$, the trend of the stock is downward concave and reflects a **accelerated loss**. 
 
 
Additionally, the magnitude of $\textbf{acceleration}$ affects how "sharp" or "broad" the curve is. Larger absolute values of $\textbf{acceleration}$ result in a steeper curve, indicating a stronger curvature, while smaller values produce a gentler curve.

In short, the quadratic coefficient $\textbf{acceleration}$ determines the overall shape of the curve (its convexity or concavity) and how pronounced that curvature is.

**Examples:**

1. **stock A**
- $\textbf{gain} = 10,  \textbf{acceleration} = 3 \to \text{accelerated gain}$


2. **stock B**
- $\textbf{gain} = 10, \textbf{acceleration} = - 3 \to \text{decelerated gain}$

We expect more positve return on **stock A** due to its $\text{accelerated gain}$. Therefore, when longing the two stocks, we should put more money on stock **A** and less money **B**

3. **Stock C**
- $\textbf{gain} = -10,  \textbf{acceleration} = 3 \to \text{decelerated loss}$
4. **Stock D**
- $\textbf{gain} = -10,  \textbf{acceleration} = -3 \to \text{accelerated loss}$

We expect more negative return on **stock D** due to its $\text{accelerated loss}$. Therefore, when shorting the two stocks, we should put more money on stock **D** and less money **C**


### Alpha Factor construction

We can now summurize that 
- When $\textbf{acceleration}(+) \times \textbf{gain}(+)$ is large, then the signal generally means to take a larger long positiom.

- When $\textbf{acceleration}(-) \times \textbf{gain}(-)$ is large, then the signal generally means to take a larger short position

Therefore, if we convert the $\textbf{gain}$ and $\textbf{acceleration}$ coefficients into ranks, then multiply them together, the product of the ranked $\textbf{gain}$ times ranked $\textbf{acceleration}$ would represent an alpha factor; a join alpha factor.

1. ranked $\textbf{gain}$ is small  and ranked $\textbf{acceleration}$ is small $\implies$  **large short** position, because the product is small.

2. ranked $\textbf{gain}$ is large  and ranked $\textbf{acceleration}$ is large $\implies$  **large long** position, because the product is large.


Note that 1. and 2. are conditional factors  because they are the combination of both momentum and convexity factors.


For the computation of the conditinal factors and the final joint factors, we need first to estimate the  coefficients  $\textbf{gain}$ and $\textbf{acceleration}$. For that, we will use a **multiple regression** where independent variables are the $t$ and $t^2$ and the dependent variable is the stock price (adjusted closing price). This regression will then give us estimates for the coefficients $\textbf{gain}$ and $\textbf{acceleration}$.

Finally, we will create our alpha factor first by converting $\textbf{gain}$ and $\textbf{acceleration}$ into ranks and them multiply the ranks together to create a join factor.